# CRC Accelerator Performance Demostration

# Simple Python experiments

In [1]:
buffer_out = []
buffer_in = []

# write the data to the accelerator
for i in range(0, 4096):
    mmio_crc.write(crc_address_offset, (test_data[0]+i))
    mmio_crc.write(crc_address_offset+0x4, crc_initial_data)
    buffer_out.append(mmio_crc.read(0x8))
#print((result))
count = 0
for n in buffer_out:
    if buffer_out.count(n)>1:
        count = count +1 
        print("Repeated number: ",n, "Count: ", result.count(n))
print(count)

NameError: name 'mmio_crc' is not defined

In [204]:
import binascii
import struct, re
import sys
import codecs

filename = "./test_image.png"
# read the image as bytes
with open(filename, 'rb') as f:
    content = f.read()
    
#print(content)
test = binascii.hexlify(content)
print(len(test))
test_b = binascii.unhexlify(test)
print(len(test_b))
vl = content.hex().ljust(8192,'0')
# print((vl))
vl_b = bytes.fromhex(vl)

#print((vl_b))

#with open("./works.png", 'wb') as f:
#    f.write(vl_b)

def return_hex(st: str):
    dummy = []
    for i in range(0, len(vl), 8):
        #print(st[i], st[i+1], st[i+2], st[i+3])
        #print(int(st[i], 16), int(st[i+1], 16), int(st[i+2], 16), int(st[i+3], 16))
        dummy.append(((int(st[i]  , 16) << 28) | \
                      (int(st[i+1], 16) << 24) | \
                      (int(st[i+2], 16) << 20) | \
                      (int(st[i+3], 16) << 16) | \
                      (int(st[i+4], 16) << 12) | \
                      (int(st[i+5], 16) <<  8) | \
                      (int(st[i+6], 16) <<  4) | \
                      (int(st[i+7], 16))))
    return dummy
    
hex_8k = return_hex(vl)

hex_img = []

for i in range(0, len(hex_8k)):
    hex_img.append(hex(hex_8k[i]).replace("0x", '').rjust(8, "0"))
    

hex_img = codecs.decode(''.join(hex_img), 'hex_codec')
with open("./works.png", 'wb') as f:
    f.write(hex_img)
#print(sys.getsizeof(hex_8k[0]))
#print(len(hex_8k))

#print(hex(hex_8k[0]))
#print(test, '\n', vl, '\n', hex(hex_8k[0]))
# temp = content + struct.pack('218x') #,hex_val)
# len(temp.
#len(binascii.hexlify(temp))

#hex(ord(re.sub('\\\\x', '', str(temp))))

7974
3987


In [209]:
def hex_to_img(hex_list, filename):
    hex_img = []

    for i in range(0, len(hex_list)):
        hex_img.append(hex(hex_list[i]).replace("0x", '').rjust(8, "0"))
    
    hex_img = codecs.decode(''.join(hex_img), 'hex_codec')
    with open(filename, 'wb') as f:
        f.write(hex_img)
        
    return(filename)

hex_to_img(hex_8k, './without_noise.png')

'./without_noise.png'

In [183]:
import codecs
data = "0xfef0babe"
bits = ""
data = data.replace("0x", "")
#for x in range(0, len(data), 2):
#  bits += chr(int(data[x:x+2], 16))

codecs.decode(data, 'hex_codec')
#print(bits)

b'\xfe\xf0\xba\xbe'

In [166]:
"""
    Function to return the list of hex values when given a file name
    input: string = Path to the file
    returns: list = the file sliced into 32bit hex elements
"""
def return_hex_list(filename):
    
    # read the file
    # the file size is 4kb for the examples we use
    with open(filename, 'rb') as f:
        content = f.read()
    
    # print the length of the bytestream
    print("length of the bytestream: ", len(content))
    
    # the list will contain 4bytes per element in hex format
    # converting the image data from bytes to hex will lead to a size increase
    temp_val = content.hex()
    len_tv = len(temp_val)
    
    # print the length of the bytestream
    print("length of the hex stream of the image: ", len_tv)    
    
    # we find the closest power of two to pad the stream
    if (len_tv > 1):
        for i in range(1, len_tv):
            if (2**i >= len_tv):
                size = 2**i
                break
    
    print("length the hex stream: ", size)
    
    st = temp_val.ljust(size,'0')
    hex_list = []
    
    for i in range(0, len(st), 8):
        #print(st[i], st[i+1], st[i+2], st[i+3])
        #print(int(st[i], 16), int(st[i+1], 16), int(st[i+2], 16), int(st[i+3], 16))
        hex_list.append(((int(st[i]  , 16) << 28) | \
                         (int(st[i+1], 16) << 24) | \
                         (int(st[i+2], 16) << 20) | \
                         (int(st[i+3], 16) << 16) | \
                         (int(st[i+4], 16) << 12) | \
                         (int(st[i+5], 16) <<  8) | \
                         (int(st[i+6], 16) <<  4) | \
                         (int(st[i+7], 16))))   
    
    # print the number of 32bit elements in the list
    print("number of 32bit elements in the list: ", len(hex_list))
    
    return hex_list

In [172]:
import random

filename = "./test_image_peppers.png"
# read the image as bytes
#with open(filename, 'rb') as f:
#    content = f.read()
hex_list = return_hex_list(filename)

def noisy(x, threshold=0.7):
    for i in range (len(x)):
        if random.random() < threshold:
            x[i] = x[i] << random.randint(0, 32) ^ random.randint(0, 2^32)
    return x

noisy_hex_list = noisy(hex_list)

length of the bytestream:  22088
length of the hex stream of the image:  44176
length the hex stream:  65536
number of 32bit elements in the list:  8192


In [173]:
"""Parameters
#----------
#image : ndarray
#    Input image data. Will be converted to float.
# type of noise added 
    's&p'       Replaces random pixels with 0 or 1.
"""

def noisy(image):
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = image
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
            for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
                  for i in image.shape]
        out[coords] = 0
        return out

# Playing with Random String. Writing to and From a 8kB BRAM

In [ ]:
# Random 8kB string

# st = "6snYXw2XyPrvwwWzpp2psrVW1pgOEvqq626gHbkf4VGN3ZRWxE6aPnorYO8JLWOD6a8oKK2gFc73zRxw20fLcjGdNWbxgBzkXaHCZdEVrq9dfhG3jZYRyAoeJdtzEWqtF0WNi3802ljrYsrJFI0287Y8o20uaYMDLAXBTvKlbHl5x0TqxYRxGuKP0v7xHUHOowNnwrUf4IcJeESBwSjXzyZxsKFy0LnWwE6v8BryiUYWMm68JzUHoXohiPMfAMS1MOVUORIXg4dlmriovlmeDbGs84lCfGU8TzjClReAi0uHoNaINB0ovjbuTI6844eYazKSk5LPOiJH9LmivYW3vMAr2Q5eXJRgAVgmcgLGEv22gNO1jShh7iVyt8BrhoJ08G1Jomedo4dbUs0LR52uXpywIbKLocs8oISDI8TtuUKVRctUaCyhIbslSNT0G3gvPmIMQfPhmoP3bxZCAH8YCPoLBIuEHf2LOh4ClF6WsMxRk3KOWtQJkDlU5ab5BSph5OHXgIM4UqUiBieVaQXuFlaC2AOKBpsONVCMwf0v9y8w4qi3l6YdGBWx3dtaP0i44M4bCqkAh88TLNh64sBSbmwm5mQvr6xcaz2BgF9iqhlaJCjG0mFBHrdy7SRtBtk5etiurd5qi9c0CbB6l0kmFxRXcBID5qLXBThsifmJXJ0RiD2WM2In559aT7L8xSaAJVnZKsHtL5fo6bpuENQFXXkGck3yeiGJZXvQu27B0BXhPUkxEJeO2mgjH6EAZ7x6l2sHpzmO8meN0b8YlVQj1n6oFXwcrH1odwhecOtKfCBeGpC6nGStoTJ3ym0TGwURJdFUCVzT0aaZ2p0Ijc78U0RJ2c9wv8iHtQjeSHZkRUjVYbbyGiPYmrhVRB4qQw1acx4QRqbvK9Os9VeoqM7MJAPowo9OQ3YQ6WlxH1WfgcDimG6mQxIk8k1aWGb04KIFqmIa5xAM1pr0lM4zVDZRO06PVpRNMaHZponIzYxnEP6dBDcbZkKDqMACswHScPahi226vVDgCBFqqyr5Y38GP7ZOmbzxvqgf7moiv1ROsv57YCYSUoTfpv2BhMzVkpHWynP3EYQPZA1Ixvp5Uy7rEDpgQ1kCMvYXNyCevVMmF7BO3w2AkNKJQI7mACyuDjrYMznd1RAejx1SIu5uKvZR9Xs0AV5cdvDUOt8DCVa3mAXSxd9Agvwu8UxWCCK2r8Xc6G08twL86OnWjpr0CY9WttAHUwttFOlQvCyFqOizfSyCKsHQbJUvuhVAXuo4NaKpY3uqVHaiqe42fST6RLFGY8w9ElxUYGURMtaQq9QYFSBbdaKTRCmz1pajtovCzxsN7AKHGwb92BBOuxufiWrqmIbafsN1xSYn1mX9ohSCViI3Mq7nwltorFB0e0xdO95JgdzC1q98buQpYC7cv7ZutH5hMD94bhwOP3AjxtXMr7YQ28UVcbrPGMBVxnK86ciPqFTkZMXt6Vgi7JplFwDK0Mxns94YRLb97TCBQJ6ijEMS86TF5g6DDlLlbdbcmGCbIcvotkfKFoXF5Fl5enACa9SqbVLorK9HdZuvcAf9PYuHJYHBgXwgW0csW9q5AySCbFzBBNkjKQb7sTfYof975jA3nBcmMWj9azxWWqXl2WyyhWRXqRyPw8vlch3DKGT9VMf01aWTWKELLZurijdQ295cx3VzBHxHjTsidJaMGFujrH5gqYkC7ya380x2Ru3EXoD9RaPAq8pW6S20SLKX1Nmke6BK1P3Zn6CnTRBb0hmDl9RqSvtFTmnv4T52rVSUdSuxyvAEwMGbIZ34CsFPahgbjLoANqMCD6BD8a7lO5ROApohzLovH9RnObe7lvlChLVojFQ9Ux3p67L7vIJvqfFnitLMBCjXn1W5yy26QSB19c3qtcwOEMGig7hzW4Ck1RRJaSU0JZZJxbHml368hK2Ss2GoRuAPfnu6bwTTtMUQonNxEzUakqgbxZjjAru7DVui5puLqejG13zTigL7SuIoF70OXG0koPCEyL2AKOLs3qPlU654pV3QU5EK8HmN8HilpNFTyBlYDpaSoGnxIeLNVGkvsCFykYjpX4DXZmSDcfZUU4Rm5FvhgAKMv8RYfyBQPS1qcM379HuEJN949tg6455HQwfV9mA4tbKD0jBIwyHnxWPcNXU9cEU84FQWkTU6hefpf9qsm9I8aqf613x5yjtGJLKfrLfZfPk6YDOZObkBIsi5uUBZ9h1GtaXgHK1lANHdBrPwMEWFJOpbnoggOyuGG39er0OXW6EdMQVA5IJPb6f0E47FuLoNyxi0HzjToHa1STKkzRWgP7o6BdMz6AvLMxzLkr8OaaIcg4vxeIUmcMtktN1ocLVoUcnVKA5mXyfL4MoYEtrta3ELRsjBjFi7EzXbb7ayXED2ODZrOzIhO5YwDUoyfjQReUBaX5U10ginUUQJgohATnvGJ154BnOYbvGIPmxNpY7eVecFUHHXhlim2ycYzciJY8xa0GPZShiUI2miWfc1I44PFBGcFJbzteSMxmBIrKCu6cAzVkLmcvEgQUt1xiFSAvTqox3NuTGwxPvcbsbxvrwYnRQrfbaFF25P3jY2h7I2ONhPwXuXKqVaodfM8nHRnZA5tmnpPHKThNPnry60FRQZII6Ua2tgEA0JYnkCPkU5voH5zEqIc4QdvlyHazMDZSgXAqu35EqmxCSESpXjcg9RFV7q2jGV5WWsGADKDaasLua7TW0ntmkpedh7HAg4OSZGalZ5NhYw9Bq1plX6IYQg6dwi8PQgKWP4dSeFFR6TU0OMPhdUfXGhGNbPRFewpVBXchTtQ9SAxb2Kx87BlHyd3qGalfYnANOQ6erI5dJLektFqNUUL4I8ZqzU1wqZJUTsvbTfr8XQlYrKnf3Fzw1lKseySbN9MymBE97lvmVOQaKEXcRWzE5xm50biILmfol8aAzqm2eOPfGE1W5P1NOcWvfLm8gZBa7GuCMEZdl8FS7ZA9L5WoqotveUwbZckD2FK1JRDvQyGeqKM7tLMbscKGvRYVwSqnpvtM4Rg992QzLRZEGEQmMFWXWOYiINHtfQwPVL6hyK3L0HlXjzMvVwcO5V38TZ4awyeY7Ff5ZoH6nMUJWrCrhUTs3JQus0G6t3BmpS69HySd75GPQmw1q10coJYSKPXRyPPL4cOJOCYMi2lZZCXyx17K900Xs4fXgCEBUPeo423PBgMPjkjnQnvleVJcj8YLIbmODkGGm5OO5rPDfJUnFrP96PvR64skww5PyqBMQVp78jOQAEL4kmNBw4WFcJQitQ6LSfGVcMDGGmnb4zcLXbPbLKPeruPQ6J2XmKwX4NgUCq2UEfCU8KOKwuFMdpqvE1qQ1dEIj49VbwLHmCOgPYVaT5bUvGlFWVrml05S2tJJ1tvxLbG9rQjhTfCBDQ6fSU78e4Z0xa1hNvOHO4sSc4U8KZWAe7PWuCdTyCiHCfYr2NYdD58Nrgwahlq5UTxyrp1cJgja2OvTP3cq1YoemhJfw2YzH7qXePTiyZZpTH1QtnZTm1KlfWcxUega29pZ5dIizLbMqQ4i1bTfzwpVOacj7UvHuQZUWwwHI0Vn0rWz4ClQwzI3KCsVeGv8LWfDhg8L0sgnay9cJuSHnbURD5H0b5DrrivXkutVoWTuFWFAHnZHje3Sz6NcgyOvUufq6JSIwwk4sGmcrLVkxNvoRev4FNpcKeJ5GQEaXWYNIsm1ytQHbVOeKMOXzrKNYCOQeFM9Pd1jHR9lfFJf4O3hAin6aWoyNKlu7fRrYoz9pYJ57h2C9cnmVovooKiuVINWKGs5nZJ1HMQTQguHXsbBClUkYNY8xjfts7sdboyqyT2xjmvBVYAsKLNOnCs6vEZnl6fYpx0G6noTtfd5KcjpKgIVc3clkVlJfaQmc0S12DJSTn5MwGu7vYMXVzEeP1EZsvXBP11Itan6KmXbk077edgHz7kfXdaTV2VE65WLat4GN45ivPAvMeDXY0twxNtnXMaE2hQDbxI57LewaGx4NmYiIBAY8hiKghWHBrPE1BPZdKyVVA4DJDO0P7Dx6qY4Wa2rLv7oMs9hqBKaMyiZoe1tohjd5ylBSysV1cBZmeWpc8c1uqJVKnpjrcOdoPNQMsOX7uIJKUoIY1Y25ORFOaGa41ogKf8ZqIzewlu07bLNk0cdblnjwjXpBJBG1NbzWEHCymEdRu9Aa49jEjwlDbqggkR2m0S9UpFWomFgDsH29vUexaHqm6NY05uBv2TKOpHvq33PRlXllScdadpwD9eme1iM0fmAoODgU5lAbjaR7mcD91tnqsqJdSqLyY2TxEHocTLI3xGUGywun8QRnXCxhDItSuPfQ0Y6mH36c4Iq9H3u5KkJfbZJqeh6xjhq20jGWRlL2xukvt7wRDzS8YSjVIEXUYF8IrFMV4GxdSOpXsjFCXz09XUPlQYfY4RLJ2cUxarQ3Nsn1pPAtMcH7ruZQJCM0C2LQF4QWH7vkQPqHL37NByzH6OxrPNoedR6ml76Jws5ViR2lokpHNMjyvXoQIZ9U5ouOh8vpwDudZOYKt4TnUTK9rPZp43LpEJusUIuZ395kOdoOYijrNiVkwQBouE7DHq8AHrxcCbZGmzCeOQUbJUY5RWNJYceSQOThOVDQSYauoOIk4jdmEue0lyTZE52HrJFJDqoxsyhZg52rfJPSHnxtVTdSqP1YDrPA4Z4mdQpU8kyvHLHc9IRnfhwLZms3eeANBSl6Hwo0IaMhvFumj3wERP9qs32RC0Aq9gfHrqaCmvSC81FY2ZoX1V25tSyEdS36dDL6RP5JPqgbucSoKshWewFaqusUxwyLtuuiJxNxSe7IylZMlEyaJSLITscXdhW1oOoqPSlT2p0TC8GmT66ZJUHgy3p00DvD7H4gQdPxrDKESiYzRFjs5H3OatYX7838o5VVU0xEgjrRGzvFq69jaUeIx2dWGQHaiDGEewQxhrmJTRHzeaAGYAlLvJdIAAGuDYpvo2MQjmmDBYEERErGVnQr1Y30cLrnx3m3KfLdqcANiVvl2ZZvS6GV4fQyeMFbiP6m6pwvw6MFf8iggHjnQ7soLc7bUcSMZo4XBUHVnCeiR8hQ1Wo3VvLIhY5ab0rQZi07Cr2sG8iLlwEw55t9wfaN7QdhKBV7vDARsEWpJEsLsmlRZD6SP0nyOuCLkUVuqHdS3bzy5Oo49Che1YrT9hVqWHrntpxB2Df4HPcToU37LHj9Nt7VvWufZ7JlKlNoTQ5WtTlUxNLT5vh3j9skbWbGke8P84MWhNaxv8hfjJXSjAMeAuImhTcOPFhUNBX08t06c71FWcm0cnj00wi6fBgtrDgiew9m0u8AQ80g28jQfQDvZaQxVbZiEdCt5cMdIzutj7XKIR0by7mdYi0VB4g1ovfHhWyISAqqeKTyiXTFgu9dqURGlF1g8gE0oPFcs6NgyA8XK140vlMkGWb0MmBrAg9j0hUCpAwMfq9veLSebaG3sL2IWDHZadiKV7Ix2jIzGyxA1U60rWxHOn4vRMuRldMRw9mLNdVRWyD4jslxAZU7DkMDJwKBD3eBx9zcZv9RGyXJEPIvEfJ768pR5pf6WrGrQkg9n7E4GTRnlFDRbWV4xvdY1iI57jg9MdPEUqi2cQR7nrBVom1aeCcePq5B7xNOo9iJGHCGwSbVAyHVJrR0aTEtJGllEBVoRaJyc524XMXUnMINieBHRwNJYsx1RvAai8IjeTtEIUUb8tjbOYOqLT3usi5BqFmKbgVyWpwrKUdLsjJlxllVMxMsmS7izINoMRxaGNbaBIZLL9c8EGO5mxBWaLz2y1qXLm24xPkAriBvx77Dow1Ir9QVdtN8zxtavLOZkjpOQbGYrdidNvhibH6kGUaFPzfloCbYs5wfDqDAIXhViXulNRBU0X8sFzldSaZhKkrClzLahprZpfFMY6umLwLoiLCUzHM3AW1CTnAnxipDLwKxPQt5g0rG3PyQUR4TBLSYbaikAUNoyE43DISvgmiDjaenMZxJcQag4AfN9rZ9yHSQy22KxI8Qhabd7oND9eDk6QQ9nRqsTsSt8eBMt6vVJdjCU85P0ckZFJqpgON8pJc1WgbltCabVyehBVOtxFciKBgdIlboQxnvzDp5vNF9jc4D0Ah6rNoz47eEglrnsatq9YMMwNEvJgWCkwXc0IIcNEpzE0WmtYaG4GAYZnmGwoDBlzySgYZngQZs8RKjtbfQv6wJkB27sNIbMGj2A3RLSn1ObYBuze9OvO0Ugxz6j2zvZycLdK4JCsAVbUMrgEMCtBkGETLizABTjZSVxR837YPD4tjSucTiwNmhklGGvJfPmfaiGIwYbQn7EQ85QXa4Kp5Y9SabDXxJm5xxU2ZNGhklF93TJjHN1qiUIfLbGkB0eiKQTG0Mufc0NbXPB3jmo5vyuIzv5dY6OLMwpMzYgonzNRRtIcLppqaJXIlZgBK6mcagzKR1VTkFvcRInQB055WKa4Sw8XZ7WXtPqMXUPNtFS5qreUEEQmonL1hgUBSJmWciYF1aEIX7Ik2XqPt2s84720ucCK4t3GGlQMU978v6BQVPCGGwSE43kAfNoLrFI5rzVSuMHgbNyrKIdb2UhM6LSG212lNt4yzjYtM6FWfDxyYQ49MDDFOzGg1e1tVIjCo9jXyHezsF2r9pgWUTRBo8ZG8Mui4yjFulmV2iossRHhBugoAuOhrNurqfbuZDFMfGMBckFjZcSDtcYTg0tuKSpZJUJEu9144PVh0w8g13nmc3Z8NyUULcK4MH2aojkicOA18PU4luB1MdMN7jO46lK0NGC2xjTkp5XP18lDJxI20ZtLtQ2vfhfcFGwd7WHZZaWaaSzgrZb9JwUCTVnRzpxHskAHjU7faJgtfwdKASu5AHHrxJpVmUlgUfavkTPvsIeDUR1WAXxOiNFJe6q1bXjNtErKvVaXCUy0udqLlMaz2SdGYwy229bjEm6W1kmQLyWMjNn3WdAcTKQ8VA1bFI69a4ovaflUqHqKEAe8ZIjwshCP1KqPiMoO3bNP0Bi5IdJCeVk6KK5wasZmRRpoN4EX94dYzWslsPgugTQOaOlut7xULlb6TsgxQD7pld1jHLlbEpT66wsknatcm0Fcvv7rxd1KGQEukTQ7JmiEAFaU9ObgTCSCBOh8cLavZ5wG7wt0IPJiM1zLKbCwLcziplzhruxq9QjkdooYvnwgOz4TsJLUlhBB9YFKbuFB5s86KgM9blFjHB1USZhb65zNl4TwqYpmbHZt3WgavxiV2FoELLDaiaC031ufHruY5OKYUmHBmuAUJS6a4IrerNo4Houtg8nnnAxYhF1UROLUaSpEwx2uzQHgCE32K7vPMFpWCfh4moHkpDjIq4njte73Pkey1hkg1WY113OPt1L3XLfPG1NsDl44ibWtgseMbRwhAYKE9p5jEVV5aMzOtFyYlyVybOiQHT7ZvsITth5Ue63AVUFVPbaoZCGgkTXJWUrR8lDfw17cOe1QFXxcIA99G1e7FZawcZdMI0LObznrNQn6Rr4ki8VQPoAo2rLgkm1b4DaLHc1yBGagDHuRJ39LRB4hewHvaLpsc92ACjBi4rP9jJKZVJ9xeZhiE2dnuIIhJPVZFdWvjtqOtfp8AasYAqwj9oYYNd82A6Er4y3CqVwlVtdH2v5BwT9Pa07ZXAWZu0wdcZOykD8SEsqDCYw7ts5XE3aWOu5Sy0zohho1mN2RzwIQEJHzpdAyhvfDKFX3kArKNpk9i0bBe1LI1bMw2nB8GXiswHgRJAKf5CZ1RQyBBQl5ZUSdiNkkTG7MuwzpCHcLcGFKr1ku9macxLJxQ658Pjv4FePem9ZIAMPOXHXwTDzX0HsxARhilmL2q6QPJdWf2e7kpTK87aa3iLsiDYmlinDcTJFkR8Y5ARxoj7k9tkix0wWuTqh8emJI0EzWm0tSUhnFi6f2wu4B3mnvcPrtdVFONq1QhQ47pIZLN6Y6WmFZglaBEdApU2GP9rsZZlZCiVyNup2AxxBC0Ce0FLVl2nXsOZmmrLdXT8yq0OGJhvrlshTkluCZEgjQlfHLa8GtDyZXSEbAdJIlWTLi5Jn2S9ZwtC0jvcnvCsHU74lo8yL500LL2hWSFbQIH6ORYuiWrGpQ9GJerynH839lLo06PuWjfNg5hKAHMqXiKUFGhQYnt9oTyfqN1AHalPULxl8"

In [ ]:
# Code to convert a string into Hex of width = 32bits

def return_hex(st: str):
    dummy = []
    for i in range(0, len(st), 4):
        dummy.append(((ord(st[i]) << 24) | (ord(st[i+1]) << 16) | (ord(st[i+2]) << 8) | ord(st[i+3])))
    return dummy
    
hex_8k = return_hex(content)

In [ ]:
#  Code to write a list of 32 bit hexadecimal values into the BRAM
for i in range (0,len(hex_8k),1):
    # print(i)
    mmio_bram.write(bram_address_offset+(i*4), hex_8k[i])

In [ ]:
# Code to read the data from the BRAM
for i in range (0,len(hex_8k),1):
    # print(i)
    mmio_bram.read(bram_address_offset+(i*4)) #, hex_8k[i])

In [ ]:
while(1):
    output_buffer.append(mmio_bram.read(i*4))
    if (output_buffer[i*2] != 0x0):
        mmio_crc.write(crc_address_offset, output_buffer[i])
        mmio_crc.write(crc_address_offset+0x4, crc_initial_data)
        output_buffer.append(mmio_crc.read(0x8))
    else:
        break
    i = i + 1
    
print('\t\t| 32bit value from image | Corresponding entry in Output Buffer | Software CRC Value\t')
print('\t\t                         |    image  |        CRC               |')

# Testing Software CRC Libraries

In [ ]:
import zlib
  
s = b'\xba\xbe\xca\xfe'
# using zlib.crc32() method
t_with_def_seed0 = zlib.crc32(s)

#t_with_seed = zlib.crc32(s, 67)
  
# to check if they are signed or unsigned?
print(hex(t_with_def_seed0))
#print(t_with_seed)

def crc32mpeg2(buf, crc=0x0):
    
    for val in buf:
        crc ^= val << 24
        for _ in range(8):
            crc = crc << 1 if (crc & 0x80000000) == 0 else (crc << 1) ^ 0x104c11db7
    return crc

crc_out = crc32mpeg2(s)
print(hex(crc_out))

In [3]:
import crcengine as crceng
params = crceng.CrcParams(0x864cfb, 24, 0xb704ce, reflect_in=False, reflect_out=False, xor_out=0)
crc_openpgp = crceng.create_from_params(params)
# this is equivalent to
crc_openpgp = crceng.create(params=params)
# invocation
result = crc_openpgp(b'123456789')
print(f'CRC=0x{result:08x}')

AttributeError: module 'crcengine' has no attribute 'CrcParams'

In [94]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=True, initCrc = 0x0, xorOut=0xffffffff)
print(hex(crc32(b'\xba\xbe\xca\xfe')))

0xf75b4227


In [30]:
init = 0xEDB88320

In [31]:
new = init << 1 or 0x1

In [29]:
print(hex(new))

0x1db710640


In [93]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=False, initCrc=0x0, xorOut=0xffffffff)
print(hex(crc32(b'\xba\xbe\xca\xfe')))

0x5a8964a8


In [12]:
init2 = 0xbabecafe
print(bin(init2))
10111010101111101100101011111110
#print(hex(b'01111111010100110111110101011101'))
new2 = 0x7f537d5d

0b10111010101111101100101011111110


In [42]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x1db710640, rev=True, initCrc=0xffffffff, xorOut=0xffffffff)
print(hex(crc32(b'0xbabecafe')))

0xffa2ac6e


In [24]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x1EDB88320, rev=False, initCrc=0xffffffff, xorOut=0xFFFFFFFF)
print(hex(crc32(b'0xbabecafe')))

ValueError: The degree of the polynomial must be 8, 16, 24, 32 or 64

In [50]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=False, initCrc=0xffffffff, xorOut=0xffffffff)
print(hex(crc32(b'0xd5d735f7')))

0x26f0072e


In [65]:
import crcmod
crc32 = crcmod.mkCrcFun(poly=0x104C11DB7, rev=False, initCrc=0xffffffff, xorOut=0x0)
print(hex(crc32(b'0xbabecafe')))

0xa8a6a95f


In [70]:
import crcmod
import crcmod.predefined
s = 
crc32_func = crcmod.predefined.mkCrcFun('crc-32-mpeg')
print(hex(crc32_func('123456789')))


TypeError: Unicode-objects must be encoded before calculating a CRC

In [73]:
import crcmod.predefined

crc32 = crcmod.predefined.Crc('crc-32')
crc32.update(b'123456789')
print(hex(crc32.crcValue))
#print(hex(crc32_func('123456789')))

0xcbf43926


In [74]:
import crcmod.predefined
crc32 = crcmod.predefined.Crc('crc-32')
crc32.update(b'0xbabecafe')
print(hex(crc32.crcValue))

0x51a9d74


In [91]:
import crcmod.predefined
s = b'\xba\xbe\xca\xfe'
#h = s.decode("hex")
#h = "0xbabecafe"
b = bytearray(s)
crc32_func = crcmod.predefined.mkCrcFun('crc-32-mpeg')
print(hex(crc32_func((b))))

0xa5769b57


In [89]:
import crcmod.predefined
crc32 = crcmod.predefined.Crc('crc-32-mpeg')
#crc32.update(b'123456789')
crc32.update(b'\xba\xbe\xca\xfe')
print(hex(crc32.crcValue))

0xa5769b57


In [ ]:
crc32 = crcmod.predefined.Crc('crc-32-mpeg')
crc32.update(b'\xba\xbe\xca\xfe')
print(hex(crc32.crcValue))
%timeit -n 10000 crc32.crcValue

# Testing Image manipulations

In [ ]:
im = cv2.imread('./test_image.png')
test_im = Image.open('./test_image_peppers.png')
print(im.shape)
display(test_im)
# print(type(noisy("s&p", im)))
# plt.imshow(noisy("s&p", im))
# plt.show

im = Image.fromarray(noisy(im))
print("why")
display(im)
im.save('test_image_peppers.png', "PNG", optimize=True, quality=80)

In [1]:
# read the image as bytes
with open(filename, 'rb') as f:
    content = f.read()#.hex()
hex_val = binascii.hexlify(content)
#print(len(content))
#print((hex_val))


NameError: name 'filename' is not defined

In [2]:
import binascii
filename = './test_image.png'
with open(filename, 'rb') as f:
    content = f.read()
hex_val = binascii.hexlify(content)
#print((hex_val))



from PIL import Image
import cv2
#import skimage.util

#im = Image.open('./sahara.jpg')
#im = cv2.imread('./sahara.jpg')
#height, width, channels = im.shape

#print(width, height, channels)

import os

print(os.path.getsize('./test_image.png'))

FileNotFoundError: [Errno 2] No such file or directory: './test_image.png'

# Other Random Stuff which probably should  have been part of the previous categories

In [ ]:
mmio_crc.write(crc_data_offset, 0x0000babe) #.rjust(32, '0'))
mmio_crc.write(crc_init_offset, 0x00000000) #.rjust(32, '0'))
mmio_crc.write(crc_sel_offset, 0x00000000) #.rjust(32, '0'))
hex(mmio_crc.read(crc_result_offset))

In [ ]:
hex(soft_crc_16('0xbabe'))